In [8]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/150.0 MB 2.9 MB/s eta 0:00:52
   ---------------------------------------- 0.3/150.0 MB 2.6 MB/s eta 0:00:57
   ---------------------------------------- 0.4/150.0 MB 2.5 MB/s eta 0:01:01
   ---------------------------------------- 0.5/150.0 MB 2.2 MB/s eta 0:01:07
   ---------------------------------------- 0.5/150.0 MB 2.2 MB/s eta 0:01:07
   ---------------------------------------- 0.7/150.0 MB 2.1 MB/s eta 0:01:12
   ---------------------------------------- 0.8/150.0 MB 2.0 MB/s eta 0:01:15
   ---------------------------------------- 0.8/150.0 MB 1.9 MB/s eta 0:01:20
   ---------------------------------------- 0.8/150.0 MB 1.9 MB/s eta 0:01:19
   ---------------------------------------- 1.0/150.0 MB 2.0 MB/s eta 0:01:16
   ---------------------------------------- 1.2/150.0 MB 2.2 MB/s eta 0:01:09



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
# from imblearn.over_sampling import SMOTE

In [16]:
# Load the dataset
df = pd.read_csv("C:/Users/harry/OneDrive/Desktop/NUS Modules/DSA4263_project/DSA4263/data/fraud_oracle_processed.csv")

In [10]:
df.head()

,Unnamed: 0,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,WeekOfMonthClaimed,Sex,Age,...,AddressChange_Claim,NumberOfCars,Year,BasePolicy,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widow,MonthClaimed,UnusualDeductible,ClaimsPerAgent
0,0,Dec,5,Wednesday,Honda,Urban,Tuesday,1,0,21,...,2,3,1994,Liability,False,True,False,1,0,15179
1,1,Jan,3,Wednesday,Honda,Urban,Monday,4,1,34,...,0,1,1994,Collision,False,True,False,1,0,15179
2,2,Oct,5,Friday,Honda,Urban,Thursday,2,1,47,...,0,1,1994,Collision,True,False,False,11,0,15179
3,3,Jun,2,Saturday,Toyota,Rural,Friday,1,1,65,...,0,1,1994,Liability,True,False,False,7,0,15179
4,4,Jan,5,Monday,Honda,Urban,Tuesday,2,0,27,...,0,1,1994,Collision,False,True,False,2,0,15179


In [17]:
# Drop irrelevant columns (if any)
df = df.drop(columns=['PolicyNumber', 'RepNumber', 'ClaimsPerAgent'], errors='ignore')

# Convert categorical columns using one-hot encoding
categorical_columns = ['Month', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed',
                       'PolicyType', 'VehicleCategory', 'BasePolicy', 'AgentType', 'Fault']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Define target variable (assuming 'FraudFound_P' is the target)
target = 'FraudFound_P'
X = df.drop(columns=[target])
y = df[target]

# Standardize numerical columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [19]:
# Train logistic regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_resampled, y_resampled)

# Predictions
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_lr)
conf_matrix = confusion_matrix(y_test, y_pred_lr)
classification_rep = classification_report(y_test, y_pred_lr)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.6615
Confusion Matrix:
[[1886 1013]
 [  31  154]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.65      0.78      2899
           1       0.13      0.83      0.23       185

    accuracy                           0.66      3084
   macro avg       0.56      0.74      0.51      3084
weighted avg       0.93      0.66      0.75      3084



In [18]:
rf_model = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=36)
rf_model.fit(X_resampled, y_resampled)
y_pred_rf = rf_model.predict(X_test)

# Evaluation
print("==== Random Forest ====")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

==== Random Forest ====
Accuracy: 0.9394
Confusion Matrix:
[[2894    5]
 [ 182    3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2899
           1       0.38      0.02      0.03       185

    accuracy                           0.94      3084
   macro avg       0.66      0.51      0.50      3084
weighted avg       0.91      0.94      0.91      3084



In [20]:
xgb_model = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb_model.fit(X_resampled, y_resampled)
y_pred_xgb = xgb_model.predict(X_test)

# Evaluation
print("==== XGBoost ====")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb))

c:\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:44:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


==== XGBoost ====
Accuracy: 0.9588
Confusion Matrix:
[[2885   14]
 [ 113   72]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2899
           1       0.84      0.39      0.53       185

    accuracy                           0.96      3084
   macro avg       0.90      0.69      0.75      3084
weighted avg       0.95      0.96      0.95      3084



In [27]:
xgb_params = {
    'n_estimators': [100, 200, 300], 
    'learning_rate': [0.05, 0.1, 0.2],  
    'max_depth': [3, 6, 8],  
    'scale_pos_weight': [1, 2, 5], 
    'subsample': [0.8, 1.0], 
    'colsample_bytree': [0.8, 1.0]  
}

In [28]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb_search = RandomizedSearchCV(xgb_model, xgb_params, scoring='f1', cv=3, n_iter=20, verbose=2, random_state=42, n_jobs=-1)
xgb_search.fit(X_resampled, y_resampled)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:45:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_cons...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [3, 6, 8],
                                        'n_estimators': [100, 200, 300],
                                        'scale_pos_weight': [1, 2, 5],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='f1', verbose=2)

In [29]:
best_xgb = xgb_search.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

print("Best XGBoost Parameters:", xgb_search.best_params_)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))

Best XGBoost Parameters: {'subsample': 0.8, 'scale_pos_weight': 1, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
XGBoost Accuracy: 0.9429312581063554
XGBoost Confusion Matrix:
 [[2896    3]
 [ 173   12]]
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      2899
           1       0.80      0.06      0.12       185

    accuracy                           0.94      3084
   macro avg       0.87      0.53      0.55      3084
weighted avg       0.94      0.94      0.92      3084



In [21]:
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_model.fit(X_resampled, y_resampled)
y_pred_lgb = lgb_model.predict(X_test)

# Evaluation
print("==== LightGBM ====")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lgb))
print("Classification Report:")
print(classification_report(y_test, y_pred_lgb))

[LightGBM] [Info] Number of positive: 11598, number of negative: 11598
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13017
[LightGBM] [Info] Number of data points in the train set: 23196, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
==== LightGBM ====
Accuracy: 0.9517
Confusion Matrix:
[[2889   10]
 [ 139   46]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      2899
           1       0.82      0.25      0.38       185

    accuracy                           0.95      3084
   macro avg       0.89      0.62      0.68      3084
weighted avg       0.95      0.95      0.94      3084



In [ ]:
lgb_params = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'max_depth': [-1, 10, 20],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}